# Anomaly Detection Framework Example

This notebook provides a walkthrough of using the anomaly detection framework in a test environment. This test environment was used as UDL's InfluxDB instance was still being setup with SkySpark data during the project. The test environment populates an instance of InfluxDB (created using Docker) with sensor data from `../../data/labelled-skyspark-data/`. The sensor data was manually downloaded from SkySpark and corresponds with five sensors used in Phase 1 model testing.

In [15]:
import time
import sys

import numpy as np
import pandas as pd

# influxdb_client is used to populate InfluxDB with the csv data
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS

Import the model package:

In [16]:
# files are contained in a sibling folder
sys.path.append("..")

import model.clean as cl
import model.model_trainer as mt
import model.model_predict as mp
from model.influx_interact import influx_class

## Step 1 - Create Local InfluxDB Instance

Copy `docker-compose.yml` located in this directory to a local directory. Then run the command `docker-compose up` from this local directory. It is recommended to increase the ram available to docker from the default of 2gb to 5gb.

Go to `http://localhost:8086/` and enter `MDS2021` as user name and `mypassword` to log in. You will need to create the `MDS2021` bucket if it is not already created.

## Step 2 - Populate InfluxDB with Sensor Data

This step will populate InfluxDB with csv files located in `../../data/labelled-skyspark-data/`. These files correspond with the Phase 1 model testing. The code presented in this section is also available in `populate_influx.py`.


In [17]:
PATH_TO_CSVS = "../../data/labelled-skyspark-data/"
CSVS_TO_LOAD = [
    "CEC_compiled_data_1b_updated.csv",
    "CEC_compiled_data_2b_updated.csv",
    "CEC_compiled_data_3b_updated.csv",
    "CEC_compiled_data_4b_updated.csv",
    "CEC_compiled_data_5b_updated.csv",
]

Set up InfluxDB connection:

In [18]:
# as setup in docker-compose.yml
token = "mytoken"
org = "UBC"
bucket = "MDS2021"

# setup InfluxDB client
client = InfluxDBClient(url="http://localhost:8086", token=token, timeout=999_000)
write_api = client.write_api(write_options=SYNCHRONOUS)

Read each csv file and write the data to InfluxDB. This sets up the sensor data in InfluxDB in the READINGS measurement mimicing how SkySpark data exists in InfluxDB. Note that only the tags/field required for anomaly detection are populated.

Important note: If the influx write times out, re-run and it should work on the second try.

In [21]:
for csv in CSVS_TO_LOAD:

    # load and set up dataframes
    df = pd.read_csv(PATH_TO_CSVS + csv, parse_dates=["Datetime"])
    df.rename(columns={"Value": "val_num"}, inplace=True)
    df.rename(columns={"ID": "uniqueID"}, inplace=True)
    df.rename(columns={"Anomaly": "AH"}, inplace=True)
    df["navName"] = "Energy"
    df["siteRef"] = "Campus Energy Centre"
    df.set_index("Datetime", drop=True, inplace=True)
    df = df.drop(["AH"], axis=1)

    print("writing: {}".format(csv))
    # write values
    write_api.write(
        bucket,
        org,
        record=df,
        data_frame_measurement_name="READINGS",
        data_frame_tag_columns=["uniqueID", "navName", "siteRef"],
    )
    time.sleep(5)

writing: CEC_compiled_data_1b_updated.csv
writing: CEC_compiled_data_2b_updated.csv
writing: CEC_compiled_data_3b_updated.csv
writing: CEC_compiled_data_4b_updated.csv
writing: CEC_compiled_data_5b_updated.csv


Look at the `df` object to see what was written to influx

In [22]:
df.head()

,val_num,uniqueID,navName,siteRef
Datetime,,,,
2020-01-01 07:45:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:00:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:15:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:30:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:45:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre


Sensor data has now been written to the InfluxDB READINGS measurement. A screenshot of what this looks like in InfluxDB is shown below.

## Step 3 - Test Anomaly Detection Model Training

This step tests model training. This would be typically run on a selected interval (for example every month) to update the anomaly detection models. A script for model training that can be used with UDL's InfluxDB instance is available in `../code/sensor_training.py`. Code that is only applicable to this test environment or differs from what would exist in `../code/sensor_training.py` is noted.

The code presented in this section is also available in `test_env_scheduled_training.py`.  

This provides the option to subset the training data for faster testing. Model training can be completed using the entire sensors record by setting this to `False`.

In [23]:
TESTING = True

Provide sensor threshold values for anomaly detection. If no value is set, the model will use the default threshold setting.

In [24]:
THRESHOLDS = {
    "Campus Energy Centre Campus HW Main Meter Power": 0.09,
    "Campus Energy Centre Boiler B-1 Exhaust O2": 0.019,
    "Campus Energy Centre Boiler B-1 Gas Pressure": 0.0725,
    "Campus Energy Centre Campus HW Main Meter Entering Water Temperature": 0.02938,
    "Campus Energy Centre Campus HW Main Meter Flow": 0.043,
}

End time to be used for model training such that data that will be predicted during Step 4 of this test environment is not used in model training In the `sensor_training.py` there is no need to set an end time as the model will train on all available data. 

In [25]:
END_TIME = 1613109600

The following code provides data removal of manually labelled anomalous data for "Campus Energy Centre Campus HW Main Meter Entering Water Temperature".

In [26]:
REMOVE_ANOMALOUS = True
REMOVE_ANOMALOUS_DATA = [
    "Campus Energy Centre Campus HW Main Meter Entering Water Temperature"
]

Specify the paths to save the model and standard scaler from the cleaning pipeline and create the InfluxDB client.

In [27]:
model_path = "./test_env_models/"
scaler_path = "./test_env_standardizers/"

# setup InfluxDB client
token = "mytoken"
org = "UBC"
bucket = "MDS2021"
url = "http://localhost:8086"

influx_conn = influx_class(
    org=org,
    url=url,
    bucket=bucket,
    token=token,
)

Read data for model training from the InfluxDB READINGS measurement

In [28]:
influx_read_df = influx_conn.make_query(
    location="Campus Energy Centre",
    measurement="READINGS",
    end=END_TIME,
)

Split the data based on uniqueID into individual sensor dataframes

In [29]:
main_bucket = cl.split_sensors(influx_read_df)

The `main_bucket` object is a dictionary with the name of the sensor as the key and then the value is another dict of data objects

In [30]:
main_bucket.keys()

dict_keys(['Campus Energy Centre Boiler B-1 Exhaust O2', 'Campus Energy Centre Boiler B-1 Gas Pressure', 'Campus Energy Centre Campus HW Main Meter Entering Water Temperature', 'Campus Energy Centre Campus HW Main Meter Flow', 'Campus Energy Centre Campus HW Main Meter Power'])

The following cell provides model training by iterating over each sensor in `main_bucket` and:

1. Removes anomalous data based on manual_anomaly labels available in the TRAINING_ANOMALY measurement
2. Standardizes the values for training and saves the standardizer
3. Subsets the data for faster training if specified in the `TESTING` variable
4. Sequences the values into windows for the LSTM-ED anomaly detection model
5. Fits the LSTEM-ED and saves the model 
6. Writes model training anomaly predictions to the TRAINING_ANOMALY Measurement model_anomaly field in InfluxDB

**Note:** 3. only applies to this test environment and would not exist in `sensor_training.py`.

In [31]:
for key, df in main_bucket.items():
    print("Training for : {}".format(key))

    # removes anomalies to only train on normal data
    if REMOVE_ANOMALOUS:
        if key in REMOVE_ANOMALOUS_DATA:
            PATH_TO_CSVS = "../../data/labelled-skyspark-data/"
            csv = "CEC_compiled_data_2b_updated.csv"
            df_with_manual_anomaly = pd.read_csv(
                PATH_TO_CSVS + csv, parse_dates=["Datetime"]
            )
            df_with_manual_anomaly["Datetime"] = pd.to_datetime(
                df_with_manual_anomaly["Datetime"], utc=True
            )
            df = df.merge(
                df_with_manual_anomaly[["Datetime", "Anomaly"]],
                how="left",
                left_on="DateTime",
                right_on="Datetime",
            )
            df = df.loc[df["Anomaly"] == False]
            df = df.drop(columns=["DateTime"], axis=1)
            am_df.rename(columns={"Anomaly": "manual_anomaly"}, inplace=True)

    # creates standardized column for each sensor in main bucket
    df["Stand_Val"] = cl.std_val_train(
        df[["Value"]],
        main_bucket[key]["ID"].any(),
        scaler_path,
    )

    if TESTING:
        df = df.tail(5000)

    # creates arrays for sliding windows
    x_train, y_train = mt.create_sequences(df["Stand_Val"], df["Stand_Val"])
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    normal_dict = cl.model_parser(df, x_train, y_train)

    threshold = THRESHOLDS[key]
    mt.fit_models(normal_dict, model_path, threshold)

    # for writing AM to influx
    am_df = normal_dict[key]["train_score_df"]
    am_df.rename(columns={"anomaly": "model_anomaly"}, inplace=True)
    am_df.rename(columns={"ID": "uniqueID"}, inplace=True)
    am_df.rename(columns={"Datetime": "DateTime"}, inplace=True)
    am_df["val_num"] = df["Value"].iloc[x_train.shape[1] :]
    # only if it hasnt already been created earlier
    if "manual_anomaly" not in set(am_df.columns):
        am_df["manual_anomaly"] = False
    am_df.set_index("DateTime", drop=True, inplace=True)
    am_df = am_df[["uniqueID", "model_anomaly", "val_num", "manual_anomaly"]]

    influx_conn.write_data(am_df, "TRAINING_ANOMALY", tags=["uniqueID", "model_anomaly", "manual_anomaly"])

Training for : Campus Energy Centre Boiler B-1 Exhaust O2
Epoch 1/100
141/141 [==============================] - 8s 31ms/step - loss: 9.1460e-04 - val_loss: 1.3933
Epoch 2/100
141/141 [==============================] - 3s 23ms/step - loss: 3.1749e-04 - val_loss: 1.4148
Epoch 3/100
141/141 [==============================] - 3s 22ms/step - loss: 2.5391e-04 - val_loss: 1.4208
Epoch 4/100
141/141 [==============================] - 3s 21ms/step - loss: 1.5189e-04 - val_loss: 1.4214
Training for : Campus Energy Centre Boiler B-1 Gas Pressure
Epoch 1/100
141/141 [==============================] - 9s 30ms/step - loss: 0.2761 - val_loss: 0.3481
Epoch 2/100
141/141 [==============================] - 3s 23ms/step - loss: 0.2147 - val_loss: 0.3155
Epoch 3/100
141/141 [==============================] - 3s 22ms/step - loss: 0.2075 - val_loss: 0.3346
Epoch 4/100
141/141 [==============================] - 3s 23ms/step - loss: 0.2049 - val_loss: 0.3429
Epoch 5/100
141/141 [=============================

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Epoch 1/100
141/141 [==============================] - 7s 26ms/step - loss: 0.2820 - val_loss: 0.2731
Epoch 2/100
141/141 [==============================] - 3s 20ms/step - loss: 0.1802 - val_loss: 0.2310
Epoch 3/100
141/141 [==============================] - 3s 19ms/step - loss: 0.1638 - val_loss: 0.1668
Epoch 4/100
141/141 [==============================] - 3s 20ms/step - loss: 0.1501 - val_loss: 0.1784
Epoch 5/100
141/141 [==============================] - 3s 22ms/step - loss: 0.1446 - val_loss: 0.1565
Epoch 6/100
141/141 [==============================] - 3s 23ms/step - loss: 0.1390 - val_loss: 0.1207
Epoch 7/100
141/141 [==============================] - 3s 24ms/step - loss: 0.1339 - val_loss: 0.1520
Epoch 8/100
141/141 [==============================] - 3s 24ms/step - loss: 0.1331 - val_loss: 0.1359
Epoch 9/100
141/141 [==============================] - 3s 22ms/step - loss: 0.1283 - val_loss: 0.1309
Training for : Campus Energy Centre Campus HW Main Meter Flow
Epoch 1/100
141/141 

The following screenshots show InfluxDB with the TRAINING_ANOMALY measurements with the model_anomaly field written from the above process.

## Step 4 - Test Anomaly Detection Predictions

This step tests anomaly predictions and includes reading recent data from InfluxDB (including the window of data required to make predictions), loading previously saved anomaly detection models, running these models on the data to provide predictions, and writing the results back to InfluxDB. This would be typically by completed on a high frequency interval (for example every minute or 5 minutes). A script for anomaly predictions that can be used with UDL's InfluxDB instance is available in `../code/sensor_predict.py`. Code that is only applicable to this test environment or differs from what would exist in `../code/sensor_predict.py` is noted.

The code presented in this section is also available in `test_env_scheduled_predictor.py`.  

First setup start and end times for the prediction data set in this testing environment. In `sensor_predict.py` END_TIME would be `now()` and START_TIME would be `now() - 1d`.

In [32]:
# END TIME FOR TRAINING SET BECOMES PREDICTING'S START TIME
START_TIME = 1613109600
END_TIME = 1613196000

Read data from InfluxDB:

In [33]:
influx_read_df = influx_conn.make_query(
    location="Campus Energy Centre",
    measurement="READINGS",
    start=START_TIME,
    end=END_TIME,
)

Split the data based on uniqueID into individual sensor dataframes

In [34]:
main_bucket = cl.split_sensors(influx_read_df)

The following cell provides predictions by iterating over each sensor in `main_bucket` and:

1. Standardizes the values for training by loading the standardizer
2. Sequences the values into windows for the LSTM-ED and other reshaping for the prediction step
3. Creates predictions for the data and returns the prediction object
4. Shapes the prediction object and write predictions to the PREDICT_ANOMALY Measurement realtime_anomaly field in InfluxDB

In [35]:
for key, df in main_bucket.items():
    main_bucket[key]["Stand_Val"] = cl.std_val_predict(
        main_bucket[key][["Value"]],
        main_bucket[key]["ID"].any(),
        scaler_path,
    )

    # creates arrays for sliding windows
    x_train, y_train = mt.create_sequences(
        main_bucket[key]["Stand_Val"], main_bucket[key]["Stand_Val"]
    )
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    timestamps = df["DateTime"].tail(len(df) - x_train.shape[1]).values
    threshold = THRESHOLDS[key]

    # predicting and prediction formatting
    pred = mp.make_prediction(
        key,
        x_train,
        timestamps,
        threshold,
        model_path,
    )
    ar_df = pd.DataFrame.from_dict(pred["data"])

    # prep for writing
    ar_df.rename(columns={"anomaly": "realtime_anomaly"}, inplace=True)
    ar_df.rename(columns={"Timestamp": "DateTime"}, inplace=True)
    ar_df["uniqueID"] = key
    ar_df.set_index("DateTime", drop=True, inplace=True)
    ar_df["val_num"] = df["Value"].tail(len(df) - x_train.shape[1]).values
    ar_df = ar_df[["uniqueID", "val_num", "realtime_anomaly"]]

    influx_conn.write_data(ar_df, "PREDICT_ANOMALY", tags=["uniqueID", "realtime_anomaly"])

Predictions are now written to InfluxDB and a screenshot of the PREDICT_ANOMALY measurement in InfluxDB is shown below:


The test environment will now have three measurements:

- READINGS: the raw data  
- TRAINING_ANOMALY: data with the manual_anomaly field and model_anomaly field generated from model training step
- PREDICT_ANOMALY: data with the realtime_anomaly field generated from the prediction step


## Step 5 - Dashboard

A template for the dashboard has been provided in this `create-test-env` directory as `cec_boiler_sensors.json`

##### To upload the dashboard json:
1) Navigate to the `Dashboards` tab on the left panel of the influxdb gui  
2) Click `Create Dashboard` in the top right  
3) Click `Import Dashboard` from the drop down  
4) Click then upload `cec_boiler_sensors.json`  
5) Click the new dashboard to view, you will have to change the start date to view data (Try 2020-12-20 to now)  

## Step 6 - Notifications

This involves creating three objects:  
1) Checks  
2) Notification Endpoints  
3) Notification Rules  

#### To create Alert Notifications

##### To Create a Check
1) Navigate to the `Alerts` tab on the left panel of the influxdb gui  
2) Click `Create` in the top right  
3) Click `Threshold Check` from the drop down  
4) Define the query to look like: 
![query](./demo_screenshots/check_01.png)
5) Configure Check as follows: 
![check](./demo_screenshots/check_02.png)
6) Click the green check box

##### To Create an Endpoint
1) Create a new slack app and copy the incoming webhook https://api.slack.com/messaging/webhooks#create_a_webhook  
2) Click `Notification Endpoints` on the middle banner  
3) Click `Create` in the top right  
4) Choose `Slack` from the drop down, name the endpoint, and paste your incoming webhook from your slack app and click Create    

##### To Create a Notification Rule  
1) Click `Notification Rules` from the middle banner  
2) Click `Create` in the top right  
3) Configure the Notification Rule to look like:
![rule](./demo_screenshots/rules_01.png)
4) Click `Create Notification Rule`

## Step 7 - Dashboard/Notification Test

Upload test data that has been flagged as anomalous to influxdb to test the notification system

The test data is set up to have 3 time stamps, now, 5 mins ago, and 10 mins ago.  
The notification system will only trigger on fresh data

In [38]:
DateTime = [int(time.time_ns()), int(time.time_ns() - 3e11), int(time.time_ns() - 6e11),]
val_num = [140.0, -40.0, 40.0]
realtime_anomaly = ["True", "True", "False"]
uniqueID = ["Campus Energy Centre Campus HW Main Meter Power"] * 3

data = {"DateTime": DateTime, "val_num":val_num, "uniqueID":uniqueID, "realtime_anomaly": realtime_anomaly}
test_realtime = pd.DataFrame(data)
test_realtime.set_index("DateTime", drop=True, inplace=True)
test_realtime.index.rename("DateTime", inplace=True)
test_realtime.head()

,val_num,uniqueID,realtime_anomaly
DateTime,,,
1623727246467240000,140.0,Campus Energy Centre Campus HW Main Meter Power,True
1623726946467239936,-40.0,Campus Energy Centre Campus HW Main Meter Power,True
1623726646467241984,40.0,Campus Energy Centre Campus HW Main Meter Power,False


In [39]:
influx_conn.write_data(test_realtime, "PREDICT_ANOMALY", tags=["uniqueID", "realtime_anomaly"])

A flagged point will appear in the check's history:
![notification](./demo_screenshots/notification_01.png)

And a notification will be pushed to your slack:
![notification](./demo_screenshots/notification_02.png) 